In [91]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
import time
import copy
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [93]:
DATA_PATH = "/content/drive/MyDrive/Dry_Bean/data/Dry_Bean_Dataset.xlsx"
dry_beans = pd.read_excel(DATA_PATH)
dry_beans.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [146]:
class DryBeanDataset(Dataset):
    def __init__(self, path):
        data = pd.read_excel(path)
        self.X = torch.Tensor(np.array(data.iloc[:, :-1], dtype=np.float32))
        self.label_encoder = LabelEncoder()
        self.standard_scaler = StandardScaler()
        self.y = torch.Tensor(self.label_encoder.fit_transform(data.iloc[:, -1]))
        self.X = torch.Tensor(self.standard_scaler.fit_transform(self.X))
        self.y_label_classes = self.label_encoder.classes_
        self.n_samples = data.shape[0]
    
    def __getitem__(self, item):
        return self.X[item], self.y[item]

    def __len__(self):
        return self.n_samples
    
    def inverse_encoder(self):
        return self.y_label_classes

In [147]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(16,50)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(50, 7)


    def forward(self, x):
        output = self.fc1(x)
        output = self.relu1(output)
        output = self.fc2(output)
        return output

In [148]:
dry_bean = DryBeanDataset(DATA_PATH)

In [149]:
X, y = dry_bean[:]

In [150]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [151]:
X_train

tensor([[-0.8175, -1.0217, -0.9929,  ...,  0.8773,  0.2669,  0.3482],
        [-0.5101, -0.4671, -0.5726,  ...,  0.3294,  0.1658,  0.0537],
        [-0.7530, -0.9509, -1.0500,  ...,  1.3828,  1.0176,  0.3483],
        ...,
        [ 1.2603,  1.4062,  1.5952,  ..., -1.2955, -0.9545, -0.2354],
        [-0.4987, -0.6714, -0.8571,  ...,  1.4155,  1.6195,  0.8617],
        [ 0.3608,  0.6632,  0.9411,  ..., -1.2211, -1.4065, -0.9976]])

In [152]:
X_test

tensor([[-0.3652, -0.5323, -0.6995,  ...,  1.1537,  1.5419,  0.9893],
        [ 0.5183,  1.0895,  0.5861,  ..., -0.6254, -0.1022, -1.9952],
        [-0.5181, -0.6840, -0.8369,  ...,  1.2638,  1.3878,  0.7263],
        ...,
        [-0.8402, -1.0839, -1.0833,  ...,  1.1801,  0.5712,  0.6621],
        [ 0.7378,  0.8947,  0.9196,  ..., -0.9058, -0.4659, -0.5394],
        [-0.4610, -0.5282, -0.4878,  ...,  0.1966,  0.0611,  0.4214]])

In [153]:
train_data = TensorDataset(X_train, y_train.type(torch.LongTensor))
test_data = TensorDataset(X_test, y_test.type(torch.LongTensor))

In [169]:
batch_size = 128
learning_rate = 0.01
num_epoch = 20

In [170]:
network = MLP()
criterion = nn.CrossEntropyLoss()
optimizer_ft = torch.optim.Adam(network.parameters(), lr=learning_rate)

In [171]:
data = {"train": train_data, "val": test_data}
dataloaders = {x: torch.utils.data.DataLoader(data[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
dataset_sizes = {x: len(data[x]) for x in ['train', 'val']}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [172]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [173]:
model_ft = train_model(network, criterion, optimizer_ft, num_epochs=num_epoch)

Epoch 0/19
----------
train Loss: 0.4675 Acc: 0.8394
val Loss: 0.2201 Acc: 0.9232

Epoch 1/19
----------
train Loss: 0.2251 Acc: 0.9185
val Loss: 0.2032 Acc: 0.9284

Epoch 2/19
----------
train Loss: 0.2085 Acc: 0.9242
val Loss: 0.1995 Acc: 0.9317

Epoch 3/19
----------
train Loss: 0.2042 Acc: 0.9238
val Loss: 0.2001 Acc: 0.9310

Epoch 4/19
----------
train Loss: 0.2031 Acc: 0.9249
val Loss: 0.2044 Acc: 0.9247

Epoch 5/19
----------
train Loss: 0.1971 Acc: 0.9266
val Loss: 0.2021 Acc: 0.9269

Epoch 6/19
----------
train Loss: 0.1952 Acc: 0.9274
val Loss: 0.2045 Acc: 0.9291

Epoch 7/19
----------
train Loss: 0.1931 Acc: 0.9288
val Loss: 0.1990 Acc: 0.9251

Epoch 8/19
----------
train Loss: 0.1943 Acc: 0.9263
val Loss: 0.2026 Acc: 0.9251

Epoch 9/19
----------
train Loss: 0.1956 Acc: 0.9301
val Loss: 0.2075 Acc: 0.9258

Epoch 10/19
----------
train Loss: 0.1949 Acc: 0.9280
val Loss: 0.1947 Acc: 0.9291

Epoch 11/19
----------
train Loss: 0.1927 Acc: 0.9289
val Loss: 0.1905 Acc: 0.9313

Ep

In [129]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [130]:
from sklearn.metrics import accuracy_score
y_predict = random_forest.predict(X_test)
accuracy_score(y_test, y_predict)

0.9250826294528094